In [ ]:
import tensorflow.keras
from unet import unet_model_3d


In [ ]:
input_shape = (1,40,40,40)
model = unet_model_3d(input_shape = input_shape, n_labels = 3, activation_name = 'relu')
#model.summary()

In [ ]:
from data import get_train_and_validation_datasets

batch_size = 2
path_images = '/home/jaume/Documentos/DL_postgraduate/final_project/sample_data/resized_training/'
path_labels = '/home/jaume/Documentos/DL_postgraduate/final_project/sample_data/resized_targets/'

train_dataset, validation_dataset, validation_images = get_train_and_validation_datasets(0.2,path_images,path_labels,resize=True,resize_shape=(100,100,80),patch=True,patch_shape=(40,40,40))
train_dataset = train_dataset.shuffle(10).batch(batch_size).take(50)
validation_dataset = validation_dataset.batch(batch_size).take(50)


In [ ]:
for image in train_dataset:
    x,y = image
    print(x.shape)
    print(y.shape)
    break

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from datetime import datetime

time = datetime.now().strftime('%Y-%m-%d_%H-%M')

model_checkpoint = ModelCheckpoint(filepath='models/best/' + time + '_model_weights.h5',
                                   monitor='val_loss',
                                   verbose=1,
                                   mode='max',
                                   save_best_only=True)

reduce_lr_plateau = ReduceLROnPlateau(monitor='val_loss', 
                                      patience=10,
                                      mode='max', 
                                      verbose=1)

tensorboard_callback = TensorBoard(log_dir='runs/' + time,
                                   write_grads=True)

callbacks = [model_checkpoint,reduce_lr_plateau,tensorboard_callback]


In [ ]:
epochs = 6
model.fit(train_dataset,
          epochs=epochs,
          validation_data=validation_dataset,
          callbacks = callbacks)

In [ ]:
model.save_weights("models/final/" + time + "_model_weights.h5")